**Cleaning For Data**
**Horizontally Add FIFA Data year by year**

In [ ]:
import pandas as pd
import openpyxl
RAW_DATA_DIR = '../data/raw'

In [12]:
df_fb = pd.read_csv(f'{RAW_DATA_DIR}/standard_stats.csv', dtype=str)
print(f"Columns in dataframe: {df_fb.columns.tolist()[:10]}...")
#df_fb.head(5)

Columns in dataframe: ['league', 'season', 'team', 'player', 'nation', 'pos', 'age', 'born', 'Playing Time_MP', 'Playing Time_Starts']...


In [13]:
# Remove players in df_fb with single names (likely data quality issues)
print(len(df_fb))
df_fb = df_fb[df_fb['player'].str.strip().str.count(' ') > 0]
print(len(df_fb))

60699
58022


In [14]:
# Create name key (unique per player per season)
# Example: Carlos Albert -> Carlos Alberto1988BRA
df_fb['name_key'] = (
    df_fb['player'] +
    df_fb['born'] + 
    df_fb['nation'] 
)
#df_fb[['name_key', 'player', 'nation', 'born']].head(5)

In [15]:
# Need to remove rows where players played for two teams in one season, make row w more minutes the sum of stats, remove other row
# Then the sheet should be ready to merge with fifa data on name_key

In [16]:
df_fifa = pd.read_csv(f'{RAW_DATA_DIR}/fifa_data.csv', dtype=str)
#df_fifa.head(10)

In [17]:
# TODO: Match fb name key -> use short name last name and mathch the 'X.' in short name to the corresponding name in long_name
# TODO: Create season column based on update_as_of year (simple if statements)
# TODO: Remove repeat columns
# TODO: Remove any players in fifa data not in fb data
# Then should be ready for horizontal concat/merge

In [18]:
#Handle Cases where you can -> otherwise use short name
# Incoroporate position into name key to differentiate players with same name and birth year/nation

In [19]:
# Last name is everything after first space in short_name
df_fifa['last_name'] = df_fifa['short_name'].str.split(' ').str[1:].str.join(' ')

# Function to find corresponding piece of long_name that matches 'X.' in short_name
# Example: 'R. Lewandowski' -> 'Robert Lewandowski'
def match_long_name(row):
    short = str(row['short_name'])
    long = str(row['long_name'])
    
    # Get the first letter from short_name (the 'R' in 'R. Lewandowski')
    first_letter = short.split(' ')[0].replace('.', '').strip().upper()
    
    # Split long_name into words
    long_words = long.split(' ')
    
    # Find the first word in long_name that starts with the same letter
    for word in long_words:
        if word and word[0].upper() == first_letter:
            # Reconstruct: first_word + last_name from short_name
            last_name = ' '.join(short.split(' ')[1:])
            return f"{word} {last_name}"
    
    # Fallback: if no match found, return the original long_name
    return long

df_fifa['matched_long_name'] = df_fifa.apply(match_long_name, axis=1)
#df_fifa[['short_name', 'long_name', 'matched_long_name']].head(10)